# ReAct Agent using LangGraph

[Yao, Shunyu, et al. "React: Synergizing reasoning and acting in language models." International Conference on Learning Representations (ICLR). 2023.](https://par.nsf.gov/servlets/purl/10451467)

In [ ]:
from langchain_dartmouth.llms import ChatDartmouthCloud
from dotenv import find_dotenv, load_dotenv

from langgraph.prebuilt import create_react_agent

load_dotenv(find_dotenv())

In [ ]:
from langchain_core.tools import tool

import requests

import os


@tool
def search_dog(
    breed: str | None = None,
    shedding: int | None = None,
    barking: int | None = None,
    protectiveness: int | None = None,
    trainability: int | None = None,
    energy: int | None = None,
    max_height: int | None = None,
    max_weight: int | None = None,
    min_height: int | None = None,
    min_weight: int | None = None,
) -> list[dict[str, str | int]]:
    """Search the database for information on a dog. Weight is in pounds, height in inches. All scales range from 1 (lowest) to 5 (highest)."""
    response = requests.get(
        url="https://api.api-ninjas.com/v1/dogs",
        params={
            "name": breed,
            "shedding": shedding,
            "barking": barking,
            "energy": energy,
            "protectiveness": protectiveness,
            "trainability": trainability,
            "max_height": max_height,
            "max_weight": max_weight,
            "min_height": min_height,
            "min_weight": min_weight,
        },
        headers={"x-api-key": os.environ["DOG_API_KEY"]},
    )
    return response.json()


tools = [search_dog]

In [ ]:
model = ChatDartmouthCloud(
    model_name="openai.gpt-4.1-mini-2025-04-14",
    max_tokens=2048,
)

In [ ]:
agent = create_react_agent(
    model,
    tools=tools,
)

In [ ]:
from IPython.display import Image

Image(agent.get_graph().draw_mermaid_png())

In [ ]:
response = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "I am looking for a large, quiet dog that is trainable",
            }
        ]
    }
)

In [ ]:
for message in response["messages"]:
    message.pretty_print()